In [1]:
using StatsBase
using Distributions
using StatsFuns
using Formatting

function clusterLikelihood(r::Float64,k::Float64,j::Int)
    c1 = lgamma(k*j+j-1)
    c2 = lgamma(k*j)
    c3 = lgamma(j+1)
    c4 = log((r/k)^(j-1))
    c5 = log((1+r/k)^(k*j+j-1))
    
    L=BigFloat(( c1 - ( c2 + c3 ) ) + ( c4 - c5 ))
    if isnan(L)
        L=-Inf
    elseif isinf(L)
        L=-Inf
    end
    return L
end;

function generateWeights(r::Float64,k::Float64,maxClusterSize::Int)
    raw=[e^clusterLikelihood(r,k,m) for m in 1:maxClusterSize]
    masked=[x for x in raw if isinf(x)==false]
    sraw=1-sum(masked)
    append!(raw,sraw)
    
    probs=[]
    for x in raw
        if isinf(x) || isnan(x)
            append!(probs,0.0)
        else
            append!(probs,x)
        end
    end
    
    return probs
end;
        
function generateClusters(r::Float64,k::Float64,N::Int,loc_weights=[]::Array)
    if length(loc_weights)==0 || sum(loc_weights)<0
        loc_weights=generateWeights(r,k,N)
    end

    clusters=sample(1:N+1, WeightVec(map(Float64,loc_weights)),N)
    if clusters[1]>N
        clusters=[N]
    else
        clusters=[clusters[i] for i in 1:length(clusters) if sum(clusters[1:i])<=N]
        append!(clusters,N-sum(clusters))
    end
    
    return clusters,loc_weights
end;

function MultivariateHypergeometricSampleBias(n::Int,D::Array,bias::Float64)
    
    N = sum(D) ## total cases = pool
    m = length(D) ## indexing
    n_otr = N-D[1] ## things left to sequence
        
    if length(D)==1
        x=[n]
    else
        x = zeros(Int64,m)
        x[1] = rand(Hypergeometric(D[1],n_otr,n)) ## starting draw
        for i = 2:m-1
            n_otr = n_otr-D[i] ## total pool reduced after sequencing/testing
            n = n-x[i-1] ## fewer resources left to sequence
            
            HGD=Hypergeometric(D[i],n_otr,n)
            weights=[w^bias for w in pdf(HGD,0:n)]
            x[i] = sample(0:n,WeightVec(weights/sum(weights))) ## biased draws from hypergeometric
        end
        x[m] = n-x[m-1]
    end
    return x
end;
    
function clusterProbBias(seqN::Int,r::Float64,k::Float64,N::Int,reps::Int,bias::Float64)
    ws=generateWeights(r,k,N) ## generate probabilities
    case_clusters=[generateClusters(r,k,N,ws)[1] for w = 1:reps] ## generate replicated clusters
    sequence_clusters=[MultivariateHypergeometricSampleBias(seqN,C,float(bias)) for C in case_clusters]
    return case_clusters,sequence_clusters
end;

In [3]:
human_sequences=174
human_cases=2000
human_clusters=[7, 2, 1, 1, 1, 1, 1, 19, 1, 1, 3, 1, 29, 1, 1, 1, 2, 1, 1, 1, 1, 1, 6, 18, 1, 1, 1, 1, 1, 2, 2, 3, 5, 2, 1, 1, 4, 1, 1, 15, 1, 4, 13, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1]

reps=2500
k=0.1
for bias in 1:3
    for r = linspace(0.5,1.1,121)
        R=string(r)
        samples=open("MERS_epi_bias_$bias.$R.1.2500r.2000c.txt","w")
#         samples=open("MERS_epi_bias_$bias.$R.1.2500r.4000c.txt","w")
#         samples=open("MERS_epi_bias_$bias.$R.5.2500r.2000c.txt","w")
        cases,sequences=clusterProbBias(human_sequences,r,k,human_cases,reps,Float64(bias))

        for (s,c) in zip(cases,sequences)
            out = @sprintf "%s\t%s\n" s c
            write(samples,out)
        end;
        close(samples)
    end;
end;